In [ ]:
!pip3 install vllm
!pip3 install autoawq
!cd /workspace/repos && git clone https://github.com/arcee-ai/mergekit.git && cd mergekit && pip install -e .

In [ ]:
# In a different terminal screen, run the vllm server

# vllm serve "Valdemardi/DeepSeek-R1-Distill-Qwen-32B-AWQ" \
# --enforce-eager \
# --max_model_len 32000 \
# --gpu-memory-utilization 0.7 \
# --port 8008 \
# --host 0.0.0.0

# Use with LlamaIndex wrapper
    
# from llama_index.llms.openai_like import OpenAILike
# llm = OpenAILike(
#             model="Valdemardi/DeepSeek-R1-Distill-Qwen-32B-AWQ",
#             temperature=0.6,
#             system_prompt=None,
#             api_base="http://0.0.0.0:8008/v1",
#             api_key="fake",
#             is_chat_model=True,
#             is_function_calling_model=True,
#             context_window=10000,
#             timeout=4000.0
#         )
# response = llm.complete("What is the meaning of life?")
# print(response.text)

# To use natively see https://docs.vllm.ai/en/latest/index.html

In [ ]:
# choose models to merge. 
# (Be sure you have enough system memory to hold all models in memory)

import gc
from transformers import AutoTokenizer, AutoModelForCausalLM

model_a_name = "Qwen/QwQ-32B"
model_b_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

model_a = AutoModelForCausalLM.from_pretrained(model_a_name)
tokenizer_a = AutoTokenizer.from_pretrained(model_a_name)
del model_a, tokenizer_a
gc.collect()

model_b = AutoModelForCausalLM.from_pretrained(model_b_name)
tokenizer_b = AutoTokenizer.from_pretrained(model_b_name)
del model_b, tokenizer_b
gc.collect()


In [ ]:
# Example yaml file
# For more examples see https://github.com/arcee-ai/mergekit

# models:
#   - model: Qwen/QwQ-32B
#     parameters:
#       weight: 0.5
#   - model: deepseek-ai/DeepSeek-R1-Distill-Qwen-32B
#     parameters:
#       weight: 0.5
# base_model: Qwen/QwQ-32B
# merge_method: dare_ties
# dtype: bfloat16

In [ ]:
# Merge models
!mergekit-yaml /workspace/repos/agentic-ai/tutorials/merge_models.yml /workspace/model_merged_daretie

In [ ]:
# Quantize model

from awq import AutoAWQForCausalLM

model_path = "/workspace/model_merged_daretie"
model = AutoAWQForCausalLM.from_pretrained(
    model_path, **{"low_cpu_mem_usage": True, "use_cache": False}
)

quant_config={
  "zero_point": True,
  "q_group_size": 128,
  "w_bit": 4,
  "version": "GEMM"
}